In [1]:
import os
from langchain_openai import ChatOpenAI
from extract import extractData
from query import query
from store import prepareData, storeData
from pinecone import Pinecone, ServerlessSpec
from answer import getAnswer

c:\Users\Yoga\Desktop\PresonalProjects\RAG-Pinecone-GPT4\RAGVenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [3]:
pdf_path = "Data/mdrNet.pdf"
index_name = "pdf-test-index" # Change this to your index name on your Pinecone account
index = pc.Index(index_name)

In [5]:
# First, upsertion step
splits = extractData(pdf_path)
data_to_upsert = prepareData(splits)
status = storeData(data_to_upsert, index)
status

Started parsing the file under job_id 761be5e5-c975-4aaf-b7f4-c901ff058c09


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 215}},
 'total_vector_count': 215}

In [6]:
# Second, Query step
query_text = ["What is the MDRNet architecture?"]
context = query(index, query_text)

# Third, generate the response
model = ChatOpenAI(openai_api_key = os.environ.get("OPENAI_API_KEY"), temperature=0, model="gpt-4")
answer = getAnswer(context, query_text, model)
answer

'The MDR-Net, or Multi-scale Dense Residual Network, is a proposed architecture inspired by the classical residual network. It is designed to take full advantage of features at different scales by extracting the liver region, and it has good generalization. The MDR-Net is a chain process from downsampling to upsampling, which fuses feature information of different scales to obtain liver regions. The features are downsampled to different scales and then undergo feature fusion. \n\nThe MDR-Net model is implemented in Python 3.7 and Tensorflow 1.8, running on a system with NVIDIA GeForce RTX 2060 12GB. The Adam optimizer is used for optimization with an initial learning rate set to 1e-4. \n\nThe overall framework of the MDR-Net network is shown in Figure 2, and the process of subpixel upsampling and inverse subpixel downsampling is shown in Figure 3.'